In [1]:
import os
import numpy as np
from scipy.constants import c
from matplotlib import pyplot as plt

data_root = os.path.join('/Users/pranshu/Desktop/radar_mmwave') #Change to your local path
cur_case = 'vitalsign1019'
cur_scenario = '50k_profile_1_gt_30pm_degree_0_round_1_baby_backseat2_face'
actual = 30
freq_cutoff = 0.42
num_tx = 10
rbw = 100
# num_tx = 20
# rbw = 10
data_path = os.path.join(data_root, cur_case, cur_scenario)
recording = np.load(os.path.join(data_path, "recording.npy"))
# calibration = np.load(os.path.join(data_path, "calibration.npy"))
config = np.load(os.path.join(data_path, "config.npy"), allow_pickle=True).item()
doppler_nfft = recording.shape[0]
print(recording.shape)
range_nfft = 400
angle_nfft = [10, 10]
data = recording
print(data.shape)


def sliding_window(data, window_size=50, step_size=3):
    """
    Apply a sliding window over the data.

    Parameters:
    - data: The input data with shape (frames, tx, rx, range).
    - window_size: The size of each window/frame slice. Default is 30.
    - step_size: The step size/increment for the sliding window. Default is 1.

    Returns:
    - A list of data slices with shape (window_size, tx, rx, range).
    """
    num_frames = data.shape[0]
    slices = []

    for start in range(0, num_frames - window_size + 1, step_size):
        end = start + window_size
        slices.append(data[start:end])

    return slices

# data is 3d --> frames, tx x rx, frequency

sliding_window_data = sliding_window(data)
print("Sliding window shape:", np.array(sliding_window_data).shape)
sliding_window_data = np.array(sliding_window_data)
error_values = 0
for i in range(0,sliding_window_data.shape[0]):
    
    data = sliding_window_data[i,:,:,:]
    processed_data = data - np.mean(data[:2,:,:], axis=0)
    range_profile = np.fft.ifft(processed_data, n=range_nfft, axis=2)
    # print(range_profile.shape)

    # Now reshape the data to frames x num_tx x num_rx = 10 x frequqnecy
    processed_data_3d = processed_data.reshape(processed_data.shape[0], num_tx, 20, processed_data.shape[-1])

    # Now fft over axis 1,2, and 3 to get AoA, AoD, and range profile

    processed_data_3d_range = np.fft.ifft(processed_data_3d, n=range_nfft, axis=3)
    processed_data_3d_range = np.fft.fft2(processed_data_3d_range, s=angle_nfft, axes=(1, 2))

    # print(processed_data_3d_range.shape)

    doppler_mean = 0.0

    for i in range(angle_nfft[0]):
        for j in range(angle_nfft[1]):
            value = processed_data_3d_range[:, i, j, :]
            doppler = np.fft.fft(np.real(value), n=doppler_nfft, axis=0)
            doppler_slide = doppler[0:len(doppler)//2,:]
            doppler_slide = np.abs(doppler_slide.T)

            freq = config['freq']
            Ts = 1/range_nfft/(freq[1]-freq[0]+1e-16) # Avoid nan checks
            time_vec = np.linspace(0,Ts*(range_nfft-1),num=range_nfft)
            dist_vec = time_vec*(c/2) # distance in meters

            d = config["sample_time"]
            pulse_repetition_frequency = 1/d
            # print("d", d)
            doppler_freq = np.fft.fftfreq(doppler_nfft,d)
            doppler_freq = doppler_freq[doppler_freq>=0]

            # freq_min = np.where(doppler_freq>=0.2)[0][0]
            freq_low = np.where(doppler_freq>=freq_cutoff)[0][0]
            freq_high = np.where(doppler_freq<=2.0)[0][-1]
            range_low = np.where(dist_vec>=0.13)[0][0]
            range_high = np.where(dist_vec<=2)[0][-1]
            
            # plt.figure(figsize=(8,6))
            extent=[doppler_freq[freq_low],doppler_freq[freq_high],dist_vec[range_low],dist_vec[range_high]]

            doppler_slide = (doppler_slide[range_low:range_high, freq_low:freq_high])

            doppler_mean += doppler_slide
            
            # plt.imshow((doppler_slide[range_low:range_high, freq_low:freq_high]) , origin='lower', extent=extent, aspect='auto')
            # plt.imshow((doppler_slide[range_low:range_high, freq_low:freq_high]) / np.max(doppler_slide[range_low:range_high, freq_low:freq_high]) , origin='lower', extent=extent, aspect='auto')

            # plt.legend()
            # plt.colorbar()
            # plt.xlabel("Doppler Frequency [Hz]")
            # plt.ylabel("Range [m]")
            # plt.title(f"{i}{j}Range-Doppler Vital Sign Heatmap: {num_tx} Tx {rbw} [Hz]")
            # plt.grid()
            # plt.show()

    doppler_mean /=100.0

    d = config["sample_time"]
    pulse_repetition_frequency = 1/d
    # print("d", d)
    doppler_freq = np.fft.fftfreq(doppler_nfft,d)
    doppler_freq = doppler_freq[doppler_freq>=0]

    freq = config['freq']
    Ts = 1/range_nfft/(freq[1]-freq[0]+1e-16) # Avoid nan checks
    time_vec = np.linspace(0,Ts*(range_nfft-1),num=range_nfft)
    dist_vec = time_vec*(c/2) # distance in meters


    freq_min = np.where(doppler_freq>=0.32)[0][0]

    freq_low = np.where(doppler_freq>=freq_cutoff)[0][0]
    freq_high = np.where(doppler_freq<=1.5)[0][-1]
    range_low = np.where(dist_vec>=0.13)[0][0]
    range_high = np.where(dist_vec<=2)[0][-1]

    # print(doppler_mean.shape)

    doppler_adjusted_mean = doppler_mean/np.max(doppler_mean)

    # print(doppler_adjusted_mean.shape)


    max_value = np.max(doppler_adjusted_mean)
    max_index = np.unravel_index(np.argmax(doppler_adjusted_mean), doppler_adjusted_mean.shape)

    # Translate indices to corresponding frequency and range
    max_freq = doppler_freq[freq_low+max_index[1]]
    max_range = dist_vec[range_low + max_index[0]]

    # Translate indices to corresponding frequency and range
    
    ground_truth = actual/60
    error = (max_freq - ground_truth)
    error = error/ground_truth
    error_values += np.absolute(error)
# print(f"Maximum Value: {max_value}")
# print(f"Frequency of Maximum Value: {max_freq} Hz")
# print(f"Range of Maximum Value: {max_range} meters")
# print("Error percentage", np.absolute(error)*100)

print("Error averaged", error_values*100/sliding_window_data.shape[0])

FileNotFoundError: [Errno 2] No such file or directory: '/Users/pranshu/Desktop/radar_mmwave/vitalsign1019/50k_profile_1_gt_30pm_degree_0_round_1_baby_backseat2_face/recording.npy'

In [ ]:
doppler_mean = 0.0

for i in range(angle_nfft[0]):
    for j in range(angle_nfft[1]):
        value = processed_data_3d_range[:, i, j, :]
        doppler = np.fft.fft(np.real(value), n=doppler_nfft, axis=0)
        doppler_slide = doppler[0:len(doppler)//2,:]
        doppler_slide = np.abs(doppler_slide.T)

        freq = config['freq']
        Ts = 1/range_nfft/(freq[1]-freq[0]+1e-16) # Avoid nan checks
        time_vec = np.linspace(0,Ts*(range_nfft-1),num=range_nfft)
        dist_vec = time_vec*(c/2) # distance in meters

        d = config["sample_time"]
        pulse_repetition_frequency = 1/d
        # print("d", d)
        doppler_freq = np.fft.fftfreq(doppler_nfft,d)
        doppler_freq = doppler_freq[doppler_freq>=0]

        # freq_min = np.where(doppler_freq>=0.2)[0][0]
        freq_low = np.where(doppler_freq>=0.35)[0][0]
        freq_high = np.where(doppler_freq<=2.0)[0][-1]
        range_low = np.where(dist_vec>=0.13)[0][0]
        range_high = np.where(dist_vec<=2)[0][-1]
        
        # plt.figure(figsize=(8,6))
        extent=[doppler_freq[freq_low],doppler_freq[freq_high],dist_vec[range_low],dist_vec[range_high]]

        doppler_slide = (doppler_slide[range_low:range_high, freq_low:freq_high])

        doppler_mean += doppler_slide
        
        # plt.imshow((doppler_slide[range_low:range_high, freq_low:freq_high]) , origin='lower', extent=extent, aspect='auto')
        # plt.imshow((doppler_slide[range_low:range_high, freq_low:freq_high]) / np.max(doppler_slide[range_low:range_high, freq_low:freq_high]) , origin='lower', extent=extent, aspect='auto')

        # plt.legend()
        # plt.colorbar()
        # plt.xlabel("Doppler Frequency [Hz]")
        # plt.ylabel("Range [m]")
        # plt.title(f"{i}{j}Range-Doppler Vital Sign Heatmap: {num_tx} Tx {rbw} [Hz]")
        # plt.grid()
        # plt.show()

doppler_mean /=100.0

d = config["sample_time"]
pulse_repetition_frequency = 1/d
print("d", d)
doppler_freq = np.fft.fftfreq(doppler_nfft,d)
doppler_freq = doppler_freq[doppler_freq>=0]

freq = config['freq']
Ts = 1/range_nfft/(freq[1]-freq[0]+1e-16) # Avoid nan checks
time_vec = np.linspace(0,Ts*(range_nfft-1),num=range_nfft)
dist_vec = time_vec*(c/2) # distance in meters


freq_min = np.where(doppler_freq>=0.25)[0][0]

freq_low = np.where(doppler_freq>=0.35)[0][0]
freq_high = np.where(doppler_freq<=1.5)[0][-1]
range_low = np.where(dist_vec>=0.13)[0][0]
range_high = np.where(dist_vec<=2)[0][-1]

print(doppler_mean.shape)

doppler_adjusted_mean = doppler_mean/np.max(doppler_mean)

print(doppler_adjusted_mean.shape)


max_value = np.max(doppler_adjusted_mean)
max_index = np.unravel_index(np.argmax(doppler_adjusted_mean), doppler_adjusted_mean.shape)

# Translate indices to corresponding frequency and range
max_freq = doppler_freq[freq_low+max_index[1]]
max_range = dist_vec[range_low + max_index[0]]

# Translate indices to corresponding frequency and range
actual = 40
ground_truth = actual/60
error = (max_freq - ground_truth)
error = error/ground_truth
print(f"Maximum Value: {max_value}")
print(f"Frequency of Maximum Value: {max_freq} Hz")
print(f"Range of Maximum Value: {max_range} meters")
print("Error percentage", error*100)









d 0.1359327507019043
(299, 66)
(299, 66)
Maximum Value: 1.0
Frequency of Maximum Value: 0.6620920972707072 Hz
Range of Maximum Value: 0.610710023779887 meters
Error percentage -0.686185409393919


In [ ]:
# doppler_mean /= 100

# d = config["sample_time"]
# pulse_repetition_frequency = 1/d
# print("d", d)
# doppler_freq = np.fft.fftfreq(doppler_nfft,d)
# doppler_freq = doppler_freq[doppler_freq>=0]

# freq = config['freq']
# Ts = 1/range_nfft/(freq[1]-freq[0]+1e-16) # Avoid nan checks
# time_vec = np.linspace(0,Ts*(range_nfft-1),num=range_nfft)
# dist_vec = time_vec*(c/2) # distance in meters

# freq_min = np.where(doppler_freq>=0.3)[0][0]
# freq_max = np.where(doppler_freq<=1.5)[0][-1]

# freq_low = np.where(doppler_freq>=0.1)[0][0]
# freq_high = np.where(doppler_freq<=2.0)[0][-1]
# range_low = np.where(dist_vec>=0.13)[0][0]
# range_high = np.where(dist_vec<=2)[0][-1]
        
# plt.figure(figsize=(8,6))
        

# extent=[doppler_freq[freq_low],doppler_freq[freq_high],dist_vec[range_low],dist_vec[range_high]]

# # i1 = 0
# # j1 = 0

# # for i in range(0,doppler_mean.shape[0]):
# #     for j in range(0, doppler_mean.shape[1]):
# #         if doppler_mean[i,j] == doppler_mean.max():
# #              i1 = i
# #              j1 = j
        

# # i1 = i1/(len(doppler_freq))


# # print(j1/doppler_nfft/d)

# # print(doppler_mean.max())

# # doppler_mean = doppler_mean/np.max(doppler_mean)

# plt.imshow( doppler_mean, origin='lower', extent=extent, aspect='auto')
# plt.legend()
# plt.colorbar()
# plt.xlabel("Doppler Frequency [Hz]")
# plt.ylabel("Range [m]")
# plt.title(f"{i}{j}Range-Doppler Vital Sign Heatmap: {num_tx} Tx {rbw} [Hz]")
# plt.grid()
# plt.show()

In [ ]:
doppler_adjusted_mean = 0.0
for i in range(angle_nfft[0]):
    for j in range(angle_nfft[1]):
        value = processed_data_3d_range[:, i, j, :]
        doppler = np.fft.fft(np.real(value), n=doppler_nfft, axis=0)
        doppler_slide = doppler[0:len(doppler)//2,:]
        doppler_slide = np.abs(doppler_slide.T)

        freq = config['freq']
        Ts = 1/range_nfft/(freq[1]-freq[0]+1e-16) # Avoid nan checks
        time_vec = np.linspace(0,Ts*(range_nfft-1),num=range_nfft)
        dist_vec = time_vec*(c/2) # distance in meters

        d = config["sample_time"]
        pulse_repetition_frequency = 1/d
        # print("d", d)
        doppler_freq = np.fft.fftfreq(doppler_nfft,d)
        doppler_freq = doppler_freq[doppler_freq>=0]


        #Toddler (1-3 years old): 24-40 breaths per minute. 
        # Preschooler (4-5 years old): 22-34 breaths per minute. 
        # School-aged child (6-12 years old): 18-30 breaths per minute
        # 12-18 breaths per minute for adults

        # So we can check maximum values in blocks with frequency greater than 0.2 Hz.

        # freq_min = np.where(doppler_freq>=0.2)[0][0]
        freq_low = np.where(doppler_freq>=0.1)[0][0]
        freq_high = np.where(doppler_freq<=2.0)[0][-1]
        range_low = np.where(dist_vec>=0.13)[0][0]
        range_high = np.where(dist_vec<=2)[0][-1]
        
        # plt.figure(figsize=(8,6))
        

        extent=[doppler_freq[freq_low],doppler_freq[freq_high],dist_vec[range_low],dist_vec[range_high]]

        doppler_slide = np.abs((doppler_slide[range_low:range_high, freq_low:freq_high]) - doppler_mean)

        doppler_adjusted_mean += doppler_slide
        
        
        # plt.imshow(doppler_slide , origin='lower', extent=extent, aspect='auto')
        # # plt.imshow((doppler_slide[range_low:range_high, freq_low:freq_high]) / np.max(doppler_slide[range_low:range_high, freq_low:freq_high]) , origin='lower', extent=extent, aspect='auto')

        # plt.legend()
        # plt.colorbar()
        # plt.xlabel("Doppler Frequency [Hz]")
        # plt.ylabel("Range [m]")
        # plt.title(f"{i}{j}Range-Doppler Vital Sign Heatmap: {num_tx} Tx {rbw} [Hz]")
        # plt.grid()
        # plt.show()

In [ ]:
doppler_adjusted_mean /=100.0

d = config["sample_time"]
pulse_repetition_frequency = 1/d
print("d", d)
doppler_freq = np.fft.fftfreq(doppler_nfft,d)
doppler_freq = doppler_freq[doppler_freq>=0]

freq = config['freq']
Ts = 1/range_nfft/(freq[1]-freq[0]+1e-16) # Avoid nan checks
time_vec = np.linspace(0,Ts*(range_nfft-1),num=range_nfft)
dist_vec = time_vec*(c/2) # distance in meters

freq_min = np.where(doppler_freq>=0.3)[0][0]
freq_max = np.where(doppler_freq<=1.5)[0][-1]

freq_low = np.where(doppler_freq>=0.1)[0][0]
freq_high = np.where(doppler_freq<=2.0)[0][-1]
range_low = np.where(dist_vec>=0.13)[0][0]
range_high = np.where(dist_vec<=2)[0][-1]
        
plt.figure(figsize=(8,6))
        
extent=[doppler_freq[freq_low],doppler_freq[freq_high],dist_vec[range_low],dist_vec[range_high]]

print(doppler_adjusted_mean.shape)

doppler_adjusted_mean = doppler_adjusted_mean/np.max(doppler_mean)

plt.imshow( doppler_adjusted_mean, origin='lower', extent=extent, aspect='auto')

max_value = np.max(doppler_mean)

# Find the indices of the maximum value
max_index = np.unravel_index(np.argmax(doppler_mean), doppler_mean.shape)

# Translate indices to corresponding frequency and range
max_freq = doppler_freq[freq_low + max_index[1]]
max_range = dist_vec[range_low + max_index[0]]

print(f"Maximum Value: {max_value}")
print(f"Frequency of Maximum Value: {max_freq} Hz")
print(f"Range of Maximum Value: {max_range} meters")

plt.legend()
plt.colorbar()
plt.xlabel("Doppler Frequency [Hz]")
plt.ylabel("Range [m]")
plt.title(f"{i}{j}Range-Doppler Vital Sign Heatmap: {num_tx} Tx {rbw} [Hz]")
plt.grid()
plt.show()

In [ ]:
#Do a norm or sum or mean over all pairs, then find the peak
#Find the range Doppler peak in every aoa aod pair, and find the frequency peak.